In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъяты лексемы с частотой выше 5.txt
    ├── 1ё_Изъяты лексемы с частотой выше 3.txt
    ├── 2а_ без сокращений.txt
    ├── 2б_Изъяты лексемы с частотой выше 100.txt
    ├── 2в_Изъяты лексемы с ч

In [2]:
ls

 __pycache__/                          herramientas.ipynb
 frecuencia.png                        parte_2.png
 frecuencia_lineas.png                 parte_2_Step1.png
 frecuencia_profesional.png            partes_discurso.png
 frecuencia_profesional_ajustada.png   utils.py
 frecuencia_step_1.png                 wandb/
 frecuencia_test_1.png                'Сокращение по частям речи'/
 funciones.py                         'сокращение по частотности'/
 generador.ipynb                      'сокращение по частотности.ipynb'


In [ ]:
import funciones
from utils import train_wrapper
import warnings
import os
import torch
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
import multiprocessing as mp
import numpy as np
import wandb
import nbformat
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gc
import re
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW
from razdel import sentenize
import numpy as np

# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
mp.set_start_method('spawn', force=True)

# import funciones
# from utils import train_wrapper
# import warnings
# import os
# import torch
# import matplotlib.pyplot as plt

# # Suprimir warnings específicos
# warnings.filterwarnings('ignore', category=UserWarning)  # Para sklearn y otros
# warnings.filterwarnings('ignore', category=FutureWarning)  # Para huggingface y transformers
# from concurrent.futures import ProcessPoolExecutor
# from tqdm import tqdm
# from contextlib import redirect_stderr
# from concurrent.futures import ProcessPoolExecutor
# import multiprocessing as mp
# import numpy as np
# mp.set_start_method('spawn', force=True)
# #os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

###############################################################################
import wandb
import nbformat
# Configurar la clave de la API como variable de entorno
os.environ["WANDB_API_KEY"] = "ca85316ed713b2425615fb3a613d7eb414c9f57f"  # Reemplaza con tu clave
# Iniciar wandb sin especificar entity (se detecta automáticamente)
wandb.init(project="model-comparison-frecuency")
#os.environ["WANDB_NOTEBOOK_NAME"] = "сокращение по частотности.ipynb"
###############################################################################


def train_wrapper(args):
    dataset, config = args  # Desempaquetar los argumentos
    result = funciones.train_and_evaluate_dataset(
        dataset['path1'],
        dataset['path2'],
        config,
        dataset['name'],
    )
    result['type'] = dataset['type']
    return result
    """
      return {
        'dataset_name': dataset_name,
        'avg_accuracy': avg_accuracy,
        'std_accuracy': std_accuracy,
        'accuracies': accuracies,
        'type': type
        }
    """

###########################################################################
def create_custom_config(dataset, model_name, model_type):
    base_config = {
        'model_name': model_name,
        'num_repeats': 6,
        'test_size': 0.2,
        'threshold': 0.5,
        'model_type': model_type
    }
    if model_type == 'gpt':
        return funciones.TrainingConfig(
            **base_config,
            max_length=512,  # GPT models typically handle longer sequences
            batch_size=4,    # Smaller batch size for GPT due to memory constraints
            epochs=2,        # Fewer epochs for few-shot learning
            learning_rate=1e-5
        )
    
    if dataset.get('freq_threshold') == 100:
        return funciones.TrainingConfig(
            **base_config,
            max_length=52,
            batch_size=128,
            epochs=3,
            learning_rate=2e-5
        )
    elif dataset.get('freq_threshold') == 49:
        return funciones.TrainingConfig(
            **base_config,
            max_length=60,
            batch_size=128,
            epochs=4,
            learning_rate=3e-5
        )
    elif dataset.get('freq_threshold') == 29:
        return funciones.TrainingConfig(
            **base_config,
            max_length=51,
            batch_size=128,
            epochs=4,
            learning_rate=3e-5
        )
    elif dataset.get('freq_threshold') == 9:
        return funciones.TrainingConfig(
            **base_config,
            max_length=45,
            batch_size=128,
            epochs=5,
            learning_rate=4e-5
        )
    elif dataset.get('freq_threshold') == 5:
        return funciones.TrainingConfig(
            **base_config,
            max_length=150,
            batch_size=32,
            epochs=6,
            learning_rate=5e-5
        )
    elif dataset.get('freq_threshold') == 3:
        return funciones.TrainingConfig(
            **base_config,
            max_length=150,
            batch_size=32,
            epochs=6,
            learning_rate=5e-5
        )
    # Second dataset naming pattern
    elif "1" in dataset.get('name', ''):
        return funciones.TrainingConfig(
            **base_config,
            max_length=60,
            batch_size=128,
            epochs=6,
            learning_rate=5e-5
        )
    elif "2" in dataset.get('name', ''):
        return funciones.TrainingConfig(
            **base_config,
            max_length=60,
            batch_size=128,
            epochs=6,
            learning_rate=5e-5
        )
    elif "3" in dataset.get('name', ''):
        return funciones.TrainingConfig(
            **base_config,
            max_length=60,
            batch_size=128,
            epochs=6,
            learning_rate=5e-5
        )
    elif "4" in dataset.get('name', ''):
        return funciones.TrainingConfig(
            **base_config,
            max_length=60,
            batch_size=128,
            epochs=6,
            learning_rate=5e-5
        )

    # Default configuration if no conditions match
    return funciones.TrainingConfig(
        **base_config,
        max_length=60,
        batch_size=128,
        epochs=4,
        learning_rate=3e-5
    )
        

models = [
#         {'model':'DeepPavlov/rubert-base-cased',
#          'name':'rubert-base',
#          'type': 'bert'},# Modelo original
#         {'model':'bert-base-multilingual-cased',
#          'name':'BERT multilingual',
#          'type': 'bert'},# BERT multilingüe
        {'model':'distilbert-base-multilingual-cased',
         'name':'distilbert-base-multilingual',
         'type': 'bert'},# Versión ligera de BERT
#         {'model':'roberta-base',
#          'name':'roberta-base', 
#         'type': 'bert'}, # RoBERTa 
        {'model': 'gpt2',
         'name': 'gpt2',
         'type': 'gpt'},  #  GPT model
#         {'model': 'facebook/opt-125m',
#          'name': 'opt-125m',
#          'type': 'gpt'}  #  GPT model

    ]

datasets = [
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100',
            'type': 'freq',
            'freq': 100
        },
        {
            'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
            'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
            'name': 'Изъяты лексемы с частотой выше 49',
            'type': 'freq',
            'freq': 49
        },
        {
            'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
            'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
            'name': 'Изъяты лексемы с частотой выше 29',
            'type': 'freq',
            'freq': 29
        },
        {
            'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
            'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
            'name': 'Изъяты лексемы с частотой выше 9',
            'type': 'freq',
            'freq': 9
        },
        {
            'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
            'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
            'name': 'Изъяты лексемы с частотой выше 5',
            'type': 'freq',
            'freq': 5
        },
        {
            'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
            'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
            'name': 'Изъяты лексемы с частотой выше 3',
            'type': 'freq',
            'freq': 3
        },
            {
            'path1': '../dataset/Сокращение по частям речи/1.Первый жанр исходная выборка.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '1.Первый жанр исходная выборка',
            'type': 'pos',
            'freq': None

        },
        {
            'path1': '../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '2.Первый жанр без клауз, включающих наречия',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '3.Первый жанр без клауз, включающих глаголы',
            'type': 'pos',
            'freq': None
        },
        {
            'path1': '../dataset/Сокращение по частям речи/4.Первый жанр без клауз, включающих глаголы и наречия.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '4.Первый жанр без клауз, включающих глаголы и наречия',
            'type': 'pos',
            'freq': None
        },
    ]

def main():

    results = []
    for model in models:
        for dataset in datasets:
            dataset_copy = dataset.copy()
            dataset_copy['name'] = f"{dataset['name']} - {model['name']}"
            config = create_custom_config(dataset, model['model'], model['type'])
            args = (dataset_copy, config)
            
            # Configurar wandb para este experimento
            run_name = f"{dataset['name']} - {model['name']}"
            wandb_run = wandb.init(
                project="model-comparison",
                name=run_name,
                config={
                    "model": model['model'],
                    "dataset": dataset['name'],
                    "type": dataset['type'],
                    "freq": dataset['freq'],
                    **config.__dict__
                },
                reinit=True
            )
            
            # Entrenar y limpiar memoria
            torch.cuda.empty_cache()
            result = train_wrapper(args)
            results.append(result) 
            
            wandb.log({
                "avg_accuracy": result['avg_accuracy'],
                "std_accuracy": result['std_accuracy']
            })
            wandb_run.finish()
        
    save_results(results, 'model_results.json')    
    wandb.finish()
    torch.cuda.empty_cache()
    
    freq_datasets = [d for d in datasets if d['type'] == 'freq']
    pos_datasets = [d for d in datasets if d['type'] == 'pos']
    freq_results = [r for r in results if r['type'] == 'freq']
    pos_results = [r for r in results if r['type'] == 'pos']
    
    if freq_datasets and freq_results:
        funciones.plot_model_performance(
            freq_datasets,
            models,
            freq_results,
            save_path='frecuencia.png'
        )
    if pos_datasets and pos_results:
        funciones.plot_model_performance(
            pos_datasets,
            models,
            pos_results,
            save_path='partes_discurso.png'
        )
    
    # Resumen de resultados
    print("\nResultados de todos los experimentos:")
    print("\n--- Frecuencia ---")
    for result in sorted(freq_results, key=lambda x: x['dataset_name']):
        print(f"{result['dataset_name']}: "
              f"{result['avg_accuracy']:.4f} ± {result['std_accuracy']:.4f}")
    print("\n--- Partes de discurso ---")
    for result in sorted(pos_results, key=lambda x: x['dataset_name']):
        print(f"{result['dataset_name']}: "
              f"{result['avg_accuracy']:.4f} ± {result['std_accuracy']:.4f}")

if __name__ == "__main__":
    main()

2025-04-14 02:02:48.382645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tuesta-lx (tuesta-lx-moscow-institute-of-physics-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


train_Изъяты лексемы с частотой выше 100 - distilbert-base-multilingual.csv
test_Изъяты лексемы с частотой выше 100 - distilbert-base-multilingual.csv


avg_accuracy,▁
std_accuracy,▁
avg_accuracy,0.60045
std_accuracy,0.04122


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


train_Изъяты лексемы с частотой выше 49 - distilbert-base-multilingual.csv
test_Изъяты лексемы с частотой выше 49 - distilbert-base-multilingual.csv


In [ ]:
#batch_size=128v

In [ ]:
#display_results()

In [ ]:
import os
import IPython

# Reiniciar el kernel
IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart()"))

# Apagar el kernel después de reiniciar
os._exit(0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def plot_model_performance(datasets, models, results, save_path='frecuencia_profesional.png'):
    """
    Genera un gráfico de barras comparando el rendimiento de modelos por dataset.

    Args:
        datasets (list): Lista de diccionarios con nombres de datasets (cada uno con clave 'name').
        models (list): Lista de diccionarios con nombres de modelos (cada uno con clave 'name').
        results (list): Lista de resultados con 'dataset_name', 'model_name' y 'avg_accuracy'.
        save_path (str): Ruta donde guardar el gráfico (por defecto 'frecuencia_profesional.png').
    """
    # Extraer nombres
    dataset_names = [d['name'] for d in datasets]
    model_names = [m['name'] for m in models]

    # Crear matriz de precisiones
    accuracies = np.zeros((len(dataset_names), len(model_names)))
    for i, dataset_name in enumerate(dataset_names):
        for j, model_name in enumerate(model_names):
            for result in results:
                if dataset_name in result['dataset_name'] and model_name in result['model_name']:
                    accuracies[i, j] = result['avg_accuracy']
                    break  # Salir del bucle una vez encontrada la precisión

    # Configuración de estilo profesional
    plt.style.use('seaborn-v0_8')

    # Crear figura con mayor resolución
    fig, ax = plt.subplots(figsize=(20, 8), dpi=100)

    # Parámetros dinámicos según cantidad de modelos
    n_models = len(model_names)
    n_datasets = len(dataset_names)

    # Ajustar el ancho de las barras en función del número de modelos y datasets
    # Reducir el ancho máximo para evitar que los números se vean demasiado juntos
    max_bar_width = 0.8 / (n_models * 1.2) if n_models > 0 else 0.15
    bar_width = min(max_bar_width, 0.1)
    index = np.arange(n_datasets)
    colors = cm.Set2(np.linspace(0, 1, n_models))  # Paleta de colores profesional

    # Espacio entre grupos de barras (datasets)
    group_spacing = 0.5

    # Calcular el ancho total ocupado por las barras de un dataset
    total_bar_width_per_dataset = n_models * bar_width

    # Calcular el desplazamiento para centrar el grupo de barras de cada dataset
    group_offset = (total_bar_width_per_dataset + (n_models - 1) * 0.02) / 2 if n_models > 1 else bar_width / 2 # Añadir un pequeño espacio entre barras del mismo dataset

    # Crear barras con mejoras visuales
    for i, (model_name, color) in enumerate(zip(model_names, colors)):
        positions = index + (i - (n_models - 1) / 2) * bar_width
        bars = ax.bar(positions,
                       accuracies[:, i],
                       bar_width,
                       label=model_name,
                       color=color,
                       edgecolor='black',
                       alpha=0.8)

        # Añadir valores encima de cada barra solo si no hay demasiados modelos
        if n_models <= 10:  # Límite para evitar clutter
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.3f}',
                        ha='center', va='bottom', fontsize=9)

    # Personalización de ejes
    plt.xlabel('Dataset', fontsize=12, fontweight='bold')
    plt.ylabel('Precisión Promedio', fontsize=12, fontweight='bold')
    plt.title('Rendimiento de Modelos por Dataset', fontsize=14, fontweight='bold', pad=20)

    # Ajustar marcas del eje X
    plt.xticks(index,
               dataset_names,
               rotation=45,
               ha='right',
               fontsize=10)

    # Añadir grid
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)

    # Mover la leyenda fuera del gráfico
    plt.legend(title='Modelos',
               title_fontsize=12,
               fontsize=10,
               loc='center left',
               bbox_to_anchor=(1.05, 0.5),
               frameon=True,
               borderaxespad=0.)

    # Ajustar límites del eje Y
    plt.ylim(0, max(accuracies.max() * 1.1, 1))

    # Ajustar layout para incluir la leyenda externa
    plt.tight_layout()

    # Guardar con alta calidad
    plt.savefig(save_path,
                dpi=300,
                bbox_inches='tight')

    # Mostrar gráfico
    plt.show()

# Ejemplo de uso:
datasets = [{'name': 'dataset1'}, {'name': 'dataset2'}, {'name': 'dataset3'}, {'name': 'dataset4'}, {'name': 'dataset5'}]
models = [{'name': 'model1'}, {'name': 'model2'}, {'name': 'model3'}, {'name': 'model4'}]
results = [
    {'dataset_name': 'dataset1', 'model_name': 'model1', 'avg_accuracy': 0.85},
    {'dataset_name': 'dataset1', 'model_name': 'model2', 'avg_accuracy': 0.88},
    {'dataset_name': 'dataset1', 'model_name': 'model3', 'avg_accuracy': 0.82},
    {'dataset_name': 'dataset1', 'model_name': 'model4', 'avg_accuracy': 0.90},
    {'dataset_name': 'dataset2', 'model_name': 'model1', 'avg_accuracy': 0.78},
    {'dataset_name': 'dataset2', 'model_name': 'model2', 'avg_accuracy': 0.81},
    {'dataset_name': 'dataset2', 'model_name': 'model3', 'avg_accuracy': 0.75},
    {'dataset_name': 'dataset2', 'model_name': 'model4', 'avg_accuracy': 0.83},
    {'dataset_name': 'dataset3', 'model_name': 'model1', 'avg_accuracy': 0.92},
    {'dataset_name': 'dataset3', 'model_name': 'model2', 'avg_accuracy': 0.95},
    {'dataset_name': 'dataset3', 'model_name': 'model3', 'avg_accuracy': 0.90},
    {'dataset_name': 'dataset3', 'model_name': 'model4', 'avg_accuracy': 0.96},
    {'dataset_name': 'dataset4', 'model_name': 'model1', 'avg_accuracy': 0.65},
    {'dataset_name': 'dataset4', 'model_name': 'model2', 'avg_accuracy': 0.68},
    {'dataset_name': 'dataset4', 'model_name': 'model3', 'avg_accuracy': 0.62},
    {'dataset_name': 'dataset4', 'model_name': 'model4', 'avg_accuracy': 0.70},
    {'dataset_name': 'dataset5', 'model_name': 'model1', 'avg_accuracy': 0.70},
    {'dataset_name': 'dataset5', 'model_name': 'model2', 'avg_accuracy': 0.73},
    {'dataset_name': 'dataset5', 'model_name': 'model3', 'avg_accuracy': 0.68},
    {'dataset_name': 'dataset5', 'model_name': 'model4', 'avg_accuracy': 0.75},
]
plot_model_performance(datasets, models, results, save_path='frecuencia_profesional_ajustada.png')